In [8]:
import os
import numpy as np
import IPython
import copy

import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets import base

import sys

PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))

from influence.inceptionModel import BinaryInceptionModel
from influence.binaryLogisticRegressionWithLBFGS import BinaryLogisticRegressionWithLBFGS
import influence.experiments
from influence.dataset import DataSet
from influence.dataset_poisoning import iterative_attack, select_examples_to_attack, get_projection_to_box_around_orig_point, generate_inception_features

from load_animals import load_animals, load_dogfish_with_koda

sns.set(color_codes=True)

In [9]:
img_side = 299
num_channels = 3
 
initial_learning_rate = 0.001 
keep_probs = None
decay_epochs = [1000, 10000]

weight_decay = 0.001

num_classes = 2
max_lbfgs_iter = 1000

num_train_ex_per_class = 900
num_test_ex_per_class = 300
batch_size = 100

dataset_name = 'dogfish_%s_%s' % (num_train_ex_per_class, num_test_ex_per_class)
data_sets = load_animals(
    num_train_ex_per_class=num_train_ex_per_class, 
    num_test_ex_per_class=num_test_ex_per_class,
    classes=['dog', 'fish'])


full_graph = tf.Graph()
top_graph = tf.Graph()


Reading animals from raw images...
class: dog
data/dog/dog_0.JPEG
data/dog/dog_1.JPEG
data/dog/dog_2.JPEG
data/dog/dog_3.JPEG
data/dog/dog_4.JPEG
data/dog/dog_5.JPEG
data/dog/dog_6.JPEG
data/dog/dog_7.JPEG
data/dog/dog_8.JPEG
data/dog/dog_9.JPEG
data/dog/dog_10.JPEG
data/dog/dog_11.JPEG
data/dog/dog_12.JPEG
data/dog/dog_13.JPEG
data/dog/dog_14.JPEG
data/dog/dog_15.JPEG
data/dog/dog_16.JPEG
data/dog/dog_17.JPEG
data/dog/dog_18.JPEG
data/dog/dog_19.JPEG
data/dog/dog_20.JPEG
data/dog/dog_21.JPEG
data/dog/dog_22.JPEG
data/dog/dog_23.JPEG
data/dog/dog_24.JPEG
data/dog/dog_25.JPEG
data/dog/dog_26.JPEG
data/dog/dog_27.JPEG
data/dog/dog_28.JPEG
data/dog/dog_29.JPEG
data/dog/dog_30.JPEG
data/dog/dog_31.JPEG
data/dog/dog_32.JPEG
data/dog/dog_33.JPEG
data/dog/dog_34.JPEG
data/dog/dog_35.JPEG
data/dog/dog_36.JPEG
data/dog/dog_37.JPEG
data/dog/dog_38.JPEG
data/dog/dog_39.JPEG
data/dog/dog_40.JPEG
data/dog/dog_41.JPEG
data/dog/dog_42.JPEG
data/dog/dog_43.JPEG
data/dog/dog_44.JPEG
data/dog/dog_45.JPE

data/dog/dog_1706.JPEG
data/dog/dog_1707.JPEG
data/dog/dog_1708.JPEG
data/dog/dog_1709.JPEG
data/dog/dog_1710.JPEG
data/dog/dog_1711.JPEG
data/dog/dog_1712.JPEG
data/dog/dog_1713.JPEG
data/dog/dog_1714.JPEG
data/dog/dog_1715.JPEG
data/dog/dog_1716.JPEG
data/dog/dog_1717.JPEG
data/dog/dog_1718.JPEG
data/dog/dog_1719.JPEG
data/dog/dog_1720.JPEG
data/dog/dog_1721.JPEG
data/dog/dog_1722.JPEG
data/dog/dog_1723.JPEG
data/dog/dog_1724.JPEG
data/dog/dog_1725.JPEG
data/dog/dog_1726.JPEG
data/dog/dog_1727.JPEG
data/dog/dog_1728.JPEG
data/dog/dog_1729.JPEG
data/dog/dog_1730.JPEG
data/dog/dog_1731.JPEG
data/dog/dog_1732.JPEG
data/dog/dog_1733.JPEG
data/dog/dog_1734.JPEG
data/dog/dog_1735.JPEG
data/dog/dog_1736.JPEG
data/dog/dog_1737.JPEG
data/dog/dog_1738.JPEG
data/dog/dog_1739.JPEG
data/dog/dog_1740.JPEG
data/dog/dog_1741.JPEG
data/dog/dog_1742.JPEG
data/dog/dog_1743.JPEG
data/dog/dog_1744.JPEG
data/dog/dog_1745.JPEG
data/dog/dog_1746.JPEG
data/dog/dog_1747.JPEG
data/dog/dog_1748.JPEG
data/dog/do

data/dog/dog_2932.JPEG
data/dog/dog_2933.JPEG
data/dog/dog_2934.JPEG
data/dog/dog_2935.JPEG
data/dog/dog_2936.JPEG
data/dog/dog_2937.JPEG
data/dog/dog_2938.JPEG
data/dog/dog_2939.JPEG
data/dog/dog_2940.JPEG
data/dog/dog_2941.JPEG
data/dog/dog_2942.JPEG
data/dog/dog_2943.JPEG
data/dog/dog_2944.JPEG
data/dog/dog_2945.JPEG
data/dog/dog_2946.JPEG
data/dog/dog_2947.JPEG
data/dog/dog_2948.JPEG
data/dog/dog_2949.JPEG
data/dog/dog_2950.JPEG
data/dog/dog_2951.JPEG
data/dog/dog_2952.JPEG
data/dog/dog_2953.JPEG
data/dog/dog_2954.JPEG
data/dog/dog_2955.JPEG
data/dog/dog_2956.JPEG
data/dog/dog_2957.JPEG
data/dog/dog_2958.JPEG
data/dog/dog_2959.JPEG
data/dog/dog_2960.JPEG
data/dog/dog_2961.JPEG
data/dog/dog_2962.JPEG
data/dog/dog_2963.JPEG
data/dog/dog_2964.JPEG
data/dog/dog_2965.JPEG
data/dog/dog_2966.JPEG
data/dog/dog_2967.JPEG
data/dog/dog_2968.JPEG
data/dog/dog_2969.JPEG
data/dog/dog_2970.JPEG
data/dog/dog_2971.JPEG
data/dog/dog_2972.JPEG
data/dog/dog_2973.JPEG
data/dog/dog_2974.JPEG
data/dog/do

data/dog/dog_4416.JPEG
data/dog/dog_4417.JPEG
data/dog/dog_4418.JPEG
data/dog/dog_4419.JPEG
data/dog/dog_4420.JPEG
data/dog/dog_4421.JPEG
data/dog/dog_4422.JPEG
data/dog/dog_4423.JPEG
data/dog/dog_4424.JPEG
data/dog/dog_4425.JPEG
data/dog/dog_4426.JPEG
data/dog/dog_4427.JPEG
data/dog/dog_4428.JPEG
data/dog/dog_4429.JPEG
data/dog/dog_4430.JPEG
data/dog/dog_4431.JPEG
data/dog/dog_4432.JPEG
data/dog/dog_4433.JPEG
data/dog/dog_4434.JPEG
data/dog/dog_4435.JPEG
data/dog/dog_4436.JPEG
data/dog/dog_4437.JPEG
data/dog/dog_4438.JPEG
data/dog/dog_4439.JPEG
data/dog/dog_4440.JPEG
data/dog/dog_4441.JPEG
data/dog/dog_4442.JPEG
data/dog/dog_4443.JPEG
data/dog/dog_4444.JPEG
data/dog/dog_4445.JPEG
data/dog/dog_4446.JPEG
data/dog/dog_4447.JPEG
data/dog/dog_4448.JPEG
data/dog/dog_4449.JPEG
data/dog/dog_4450.JPEG
data/dog/dog_4451.JPEG
data/dog/dog_4452.JPEG
data/dog/dog_4453.JPEG
data/dog/dog_4454.JPEG
data/dog/dog_4455.JPEG
data/dog/dog_4456.JPEG
data/dog/dog_4457.JPEG
data/dog/dog_4458.JPEG
data/dog/do

data/dog/dog_5289.JPEG
data/dog/dog_5290.JPEG
data/dog/dog_5291.JPEG
data/dog/dog_5292.JPEG
data/dog/dog_5293.JPEG
data/dog/dog_5294.JPEG
data/dog/dog_5295.JPEG
data/dog/dog_5296.JPEG
data/dog/dog_5297.JPEG
data/dog/dog_5298.JPEG
data/dog/dog_5299.JPEG
data/dog/dog_5300.JPEG
data/dog/dog_5301.JPEG
data/dog/dog_5302.JPEG
data/dog/dog_5303.JPEG
data/dog/dog_5304.JPEG
data/dog/dog_5305.JPEG
data/dog/dog_5306.JPEG
data/dog/dog_5307.JPEG
data/dog/dog_5308.JPEG
data/dog/dog_5309.JPEG
data/dog/dog_5310.JPEG
data/dog/dog_5311.JPEG
data/dog/dog_5312.JPEG
data/dog/dog_5313.JPEG
data/dog/dog_5314.JPEG
data/dog/dog_5315.JPEG
data/dog/dog_5316.JPEG
data/dog/dog_5317.JPEG
data/dog/dog_5318.JPEG
data/dog/dog_5319.JPEG
data/dog/dog_5320.JPEG
data/dog/dog_5321.JPEG
data/dog/dog_5322.JPEG
data/dog/dog_5323.JPEG
data/dog/dog_5324.JPEG
data/dog/dog_5325.JPEG
data/dog/dog_5326.JPEG
data/dog/dog_5327.JPEG
data/dog/dog_5328.JPEG
data/dog/dog_5329.JPEG
data/dog/dog_5330.JPEG
data/dog/dog_5331.JPEG
data/dog/do

...

data/dog/dog_14574.JPEG
data/dog/dog_14575.JPEG
data/dog/dog_14576.JPEG
data/dog/dog_14577.JPEG
data/dog/dog_14578.JPEG
data/dog/dog_14579.JPEG
data/dog/dog_14580.JPEG
data/dog/dog_14581.JPEG
data/dog/dog_14582.JPEG
data/dog/dog_14583.JPEG
data/dog/dog_14584.JPEG
data/dog/dog_14585.JPEG
data/dog/dog_14586.JPEG
data/dog/dog_14587.JPEG
data/dog/dog_14588.JPEG
data/dog/dog_14589.JPEG
data/dog/dog_14590.JPEG
data/dog/dog_14591.JPEG
data/dog/dog_14592.JPEG
data/dog/dog_14593.JPEG
data/dog/dog_14594.JPEG
data/dog/dog_14595.JPEG
data/dog/dog_14596.JPEG
data/dog/dog_14597.JPEG
data/dog/dog_14598.JPEG
data/dog/dog_14599.JPEG
data/dog/dog_14600.JPEG
data/dog/dog_14601.JPEG
data/dog/dog_14602.JPEG
data/dog/dog_14603.JPEG
data/dog/dog_14604.JPEG
data/dog/dog_14605.JPEG
data/dog/dog_14606.JPEG
data/dog/dog_14607.JPEG
data/dog/dog_14608.JPEG
data/dog/dog_14609.JPEG
data/dog/dog_14610.JPEG
data/dog/dog_14611.JPEG
data/dog/dog_14612.JPEG
data/dog/dog_14613.JPEG
data/dog/dog_14614.JPEG
data/dog/dog_146

data/dog/dog_16175.JPEG
data/dog/dog_16176.JPEG
data/dog/dog_16177.JPEG
data/dog/dog_16178.JPEG
data/dog/dog_16179.JPEG
data/dog/dog_16180.JPEG
data/dog/dog_16181.JPEG
data/dog/dog_16182.JPEG
data/dog/dog_16183.JPEG
data/dog/dog_16184.JPEG
data/dog/dog_16185.JPEG
data/dog/dog_16186.JPEG
data/dog/dog_16187.JPEG
data/dog/dog_16188.JPEG
data/dog/dog_16189.JPEG
data/dog/dog_16190.JPEG
data/dog/dog_16191.JPEG
data/dog/dog_16192.JPEG
data/dog/dog_16193.JPEG
data/dog/dog_16194.JPEG
data/dog/dog_16195.JPEG
data/dog/dog_16196.JPEG
data/dog/dog_16197.JPEG
data/dog/dog_16198.JPEG
data/dog/dog_16199.JPEG
data/dog/dog_16200.JPEG
data/dog/dog_16201.JPEG
data/dog/dog_16202.JPEG
data/dog/dog_16203.JPEG
data/dog/dog_16204.JPEG
data/dog/dog_16205.JPEG
data/dog/dog_16206.JPEG
data/dog/dog_16207.JPEG
data/dog/dog_16208.JPEG
data/dog/dog_16209.JPEG
data/dog/dog_16210.JPEG
data/dog/dog_16211.JPEG
data/dog/dog_16212.JPEG
data/dog/dog_16213.JPEG
data/dog/dog_16214.JPEG
data/dog/dog_16215.JPEG
data/dog/dog_162

data/dog/dog_17926.JPEG
data/dog/dog_17927.JPEG
data/dog/dog_17928.JPEG
data/dog/dog_17929.JPEG
data/dog/dog_17930.JPEG
data/dog/dog_17931.JPEG
data/dog/dog_17932.JPEG
data/dog/dog_17933.JPEG
data/dog/dog_17934.JPEG
data/dog/dog_17935.JPEG
data/dog/dog_17936.JPEG
data/dog/dog_17937.JPEG
data/dog/dog_17938.JPEG
data/dog/dog_17939.JPEG
data/dog/dog_17940.JPEG
data/dog/dog_17941.JPEG
data/dog/dog_17942.JPEG
data/dog/dog_17943.JPEG
data/dog/dog_17944.JPEG
data/dog/dog_17945.JPEG
data/dog/dog_17946.JPEG
data/dog/dog_17947.JPEG
data/dog/dog_17948.JPEG
data/dog/dog_17949.JPEG
data/dog/dog_17950.JPEG
data/dog/dog_17951.JPEG
data/dog/dog_17952.JPEG
data/dog/dog_17953.JPEG
data/dog/dog_17954.JPEG
data/dog/dog_17955.JPEG
data/dog/dog_17956.JPEG
data/dog/dog_17957.JPEG
data/dog/dog_17958.JPEG
data/dog/dog_17959.JPEG
data/dog/dog_17960.JPEG
data/dog/dog_17961.JPEG
data/dog/dog_17962.JPEG
data/dog/dog_17963.JPEG
data/dog/dog_17964.JPEG
data/dog/dog_17965.JPEG
data/dog/dog_17966.JPEG
data/dog/dog_179

data/dog/dog_19926.JPEG
data/dog/dog_19927.JPEG
data/dog/dog_19928.JPEG
data/dog/dog_19929.JPEG
data/dog/dog_19930.JPEG
data/dog/dog_19931.JPEG
data/dog/dog_19932.JPEG
data/dog/dog_19933.JPEG
data/dog/dog_19934.JPEG
data/dog/dog_19935.JPEG
data/dog/dog_19936.JPEG
data/dog/dog_19937.JPEG
data/dog/dog_19938.JPEG
data/dog/dog_19939.JPEG
data/dog/dog_19940.JPEG
data/dog/dog_19941.JPEG
data/dog/dog_19942.JPEG
data/dog/dog_19943.JPEG
data/dog/dog_19944.JPEG
data/dog/dog_19945.JPEG
data/dog/dog_19946.JPEG
data/dog/dog_19947.JPEG
data/dog/dog_19948.JPEG
data/dog/dog_19949.JPEG
data/dog/dog_19950.JPEG
data/dog/dog_19951.JPEG
data/dog/dog_19952.JPEG
data/dog/dog_19953.JPEG
data/dog/dog_19954.JPEG
data/dog/dog_19955.JPEG
data/dog/dog_19956.JPEG
data/dog/dog_19957.JPEG
data/dog/dog_19958.JPEG
data/dog/dog_19959.JPEG
data/dog/dog_19960.JPEG
data/dog/dog_19961.JPEG
data/dog/dog_19962.JPEG
data/dog/dog_19963.JPEG
data/dog/dog_19964.JPEG
data/dog/dog_19965.JPEG
data/dog/dog_19966.JPEG
data/dog/dog_199

KeyboardInterrupt: 

In [ ]:
print('*** Full:')
with full_graph.as_default():
    full_model_name = '%s_inception_wd-%s' % (dataset_name, weight_decay)
    full_model = BinaryInceptionModel(
        img_side=img_side,
        num_channels=num_channels,
        weight_decay=weight_decay,
        num_classes=num_classes, 
        batch_size=batch_size,
        data_sets=data_sets,
        initial_learning_rate=initial_learning_rate,
        keep_probs=keep_probs,
        decay_epochs=decay_epochs,
        mini_batch=True,
        train_dir='output',
        log_dir='log',
        model_name=full_model_name)

    for data_set, label in [
        (data_sets.train, 'train'),
        (data_sets.test, 'test')]:

        inception_features_path = 'output/%s_inception_features_new_%s.npz' % (dataset_name, label)
        if not os.path.exists(inception_features_path):

            print('Inception features do not exist. Generating %s...' % label)
            data_set.reset_batch()
            
            num_examples = data_set.num_examples
            assert num_examples % batch_size == 0

            inception_features_val = generate_inception_features(
                full_model, 
                data_set.x, 
                data_set.labels, 
                batch_size=batch_size)
            
            np.savez(
                inception_features_path, 
                inception_features_val=inception_features_val,
                labels=data_set.labels)


train_f = np.load('output/%s_inception_features_new_train.npz' % dataset_name)
train = DataSet(train_f['inception_features_val'], train_f['labels'])
test_f = np.load('output/%s_inception_features_new_test.npz' % dataset_name)
test = DataSet(test_f['inception_features_val'], test_f['labels'])

validation = None

inception_data_sets = base.Datasets(train=train, validation=validation, test=test)

print('*** Top:')
with top_graph.as_default():
    top_model_name = '%s_inception_onlytop_wd-%s' % (dataset_name, weight_decay)
    input_dim = 2048
    top_model = BinaryLogisticRegressionWithLBFGS(
        input_dim=input_dim,
        weight_decay=weight_decay,
        max_lbfgs_iter=max_lbfgs_iter,
        num_classes=num_classes, 
        batch_size=batch_size,
        data_sets=inception_data_sets,
        initial_learning_rate=initial_learning_rate,
        keep_probs=keep_probs,
        decay_epochs=decay_epochs,
        mini_batch=False,
        train_dir='output',
        log_dir='log',
        model_name=top_model_name)
    top_model.train()
    weights = top_model.sess.run(top_model.weights)
    orig_weight_path = 'output/inception_weights_%s.npy' % top_model_name
    np.save(orig_weight_path, weights)


with full_graph.as_default():
    full_model.load_weights_from_disk(orig_weight_path, do_save=False, do_check=True)
    full_model.reset_datasets()


### Create poisoned dataset
print('Creating poisoned dataset...')

step_size = 0.02

num_train = len(top_model.data_sets.train.labels)
num_test = len(top_model.data_sets.test.labels)
max_num_to_poison = 10
loss_type = 'normal_loss'


### Try attacking each test example individually

orig_X_train = np.copy(data_sets.train.x)
orig_Y_train = np.copy(data_sets.train.labels)

for test_idx in range(num_test):

    print('****** Attacking test_idx %s ******' % test_idx)
    test_description = test_idx

    # If this has already been successfully attacked, skip
    filenames = [filename for filename in os.listdir('./output') if (
        (('%s_attack_%s_testidx-%s_trainidx-' % (full_model.model_name, loss_type, test_description)) in filename) and        
        (filename.endswith('stepsize-%s_proj_final.npz' % step_size)))]
        # and (('stepsize-%s_proj_final.npz' % step_size) in filename))] # Check all step sizes        
    if len(filenames) > 0:
        print('test_idx %s has already been successfully attacked. Skipping...')
        continue


    # Use top model to quickly generate inverse HVP
    with top_graph.as_default():
        #CHANGE
        top_model.get_influence_on_test_loss(
            [test_idx], 
            [0],        
            test_description=test_description,
            force_refresh=True)
    copyfile(
        'output/%s-cg-normal_loss-test-%s.npz' % (top_model_name, test_description),
        'output/%s-cg-normal_loss-test-%s.npz' % (full_model_name, test_description))

    # Use full model to select indices to poison
    with full_graph.as_default():
        #CHANGE
        grad_influence_wrt_input_val = full_model.get_grad_of_influence_wrt_input(
            np.arange(num_train), 
            [test_idx], 
            force_refresh=False,
            test_description=test_description,
            loss_type=loss_type)    
        all_indices_to_poison = select_examples_to_attack(
            full_model, 
            max_num_to_poison, 
            grad_influence_wrt_input_val,
            step_size=step_size)

    for num_to_poison in [0.1, 1.2, 2, 3, 5, 10]:
        # If we're just attacking one training example, try attacking the first one and also the second one separately
        if num_to_poison == 0.1:
            indices_to_poison = all_indices_to_poison[0:1]
        elif num_to_poison == 1.2:
            indices_to_poison = all_indices_to_poison[1:2]
        else:
            indices_to_poison = all_indices_to_poison[:num_to_poison]
        orig_X_train_subset = np.copy(full_model.data_sets.train.x[indices_to_poison, :])
        orig_X_train_inception_features_subset = np.copy(top_model.data_sets.train.x[indices_to_poison, :])

        project_fn = get_projection_to_box_around_orig_point(orig_X_train_subset, box_radius_in_pixels=0.5)

        #maybe change?
        attack_success = iterative_attack(top_model, full_model, top_graph, full_graph, project_fn, [test_idx], test_description=test_description, 
            indices_to_poison=indices_to_poison,
            num_iter=100,
            step_size=step_size,
            save_iter=100,
            loss_type=loss_type,
            early_stop=0.5)

        with full_graph.as_default():
            full_model.update_train_x_y(orig_X_train, orig_Y_train)
            full_model.load_weights_from_disk(orig_weight_path, do_save=False, do_check=False)
        with top_graph.as_default():
            X_train = top_model.data_sets.train.x
            X_train[indices_to_poison, :] = orig_X_train_inception_features_subset
            top_model.update_train_x(X_train)
            top_model.train()

        if attack_success:
            break